# VacationPy
---

## Data Analyst

Vidal Garcia Martinez

#### Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,70.6333,147.9167,-31.94,96,100,2.44,RU,02/08/2023 22:42:43
1,1,verkhoyansk,67.5447,133.3850,-43.10,95,75,0.33,RU,02/08/2023 22:46:23
2,2,albany,42.6001,-73.9662,0.15,80,81,1.23,US,02/08/2023 22:41:57
3,3,cidreira,-30.1811,-50.2056,24.49,83,24,8.46,BR,02/08/2023 22:40:27
4,4,vaini,-21.2000,-175.2000,29.09,79,75,2.06,TO,02/08/2023 22:41:55


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

# Configure the map plot
map_plot_hum = city_data_df.hvplot.points("Lng","Lat",geo = True,tiles = "OSM",
                                          frame_width = 700,frame_height = 500,
                                          size = "Humidity",color = "City", scale=0.5)

# Display the map
map_plot_hum

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [39]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df['Max Temp'] < 30) & (city_data_df['Max Temp'] > 20) 
                              & (city_data_df['Humidity'] < 50) & (city_data_df['Humidity'] > 30)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,busselton,-33.6500,115.3333,27.94,35,0,1.30,AU,02/08/2023 22:41:54
56,56,tezu,27.9167,96.1667,22.99,33,71,1.44,IN,02/08/2023 22:46:43
71,71,ayolas,-27.4000,-56.9000,27.50,32,82,3.36,PY,02/08/2023 22:46:47
99,99,tura,25.5198,90.2201,22.98,39,48,1.31,IN,02/08/2023 22:42:15
155,155,trelew,-43.2490,-65.3051,22.82,41,52,11.70,AR,02/08/2023 22:47:08


### Step 3: Create a new DataFrame called `hotel_df`.

In [40]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather.filter(['City', 'Country', 'Lat','Lng','Humidity'],axis=1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] =''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,busselton,AU,-33.6500,115.3333,35,
56,tezu,IN,27.9167,96.1667,33,
71,ayolas,PY,-27.4000,-56.9000,32,
99,tura,IN,25.5198,90.2201,39,
155,trelew,AR,-43.2490,-65.3051,41,
211,yaan,NG,7.3833,8.5667,37,
213,remanso,BR,-9.6179,-42.0831,46,
253,fram,PY,-26.9833,-55.8833,37,
263,banda aceh,IN,25.4833,80.3333,32,
320,veraval,IN,20.9000,70.3667,33,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)